In [12]:
from qcg.generators import gen_supremacy, gen_hwea
from qiskit.circuit.classicalregister import ClassicalRegister
from qiskit import QuantumCircuit
from qiskit import Aer, IBMQ, execute
import numpy as np
from scipy.stats import wasserstein_distance

def reverseBits(num,bitSize): 
    binary = bin(num)
    reverse = binary[-1:1:-1] 
    reverse = reverse + (bitSize - len(reverse))*'0'
    return int(reverse,2)

num_shots = int(1e5)
circ = gen_supremacy(2,4,8,order='75601234')

backend = Aer.get_backend('statevector_simulator')
job = execute(circ, backend=backend)
result = job.result()
outputstate = result.get_statevector(circ)
outputstate_ordered = [0 for sv in outputstate]
for i, sv in enumerate(outputstate):
    reverse_i = reverseBits(i,len(circ.qubits))
    outputstate_ordered[reverse_i] = sv
output_prob = [np.power(np.absolute(x),2) for x in outputstate_ordered]
print(sum(output_prob))



c = ClassicalRegister(len(circ.qubits), 'c')
meas = QuantumCircuit(circ.qregs[0], c)
meas.barrier(circ.qubits)
meas.measure(circ.qubits,c)
qc = circ+meas
backend = Aer.get_backend('qasm_simulator')
job_sim = execute(qc, backend, shots=num_shots)
result = job_sim.result()
counts = result.get_counts(qc)
prob_ordered = [0 for x in range(np.power(2,len(circ.qubits)))]
for state in counts:
    reversed_state = reverseBits(int(state,2),len(circ.qubits))
    prob_ordered[reversed_state] = counts[state]/num_shots
print(sum(prob_ordered))

print(wasserstein_distance(output_prob,prob_ordered))

0.999999999999998
1.0000000000000004
6.423106505955482e-05
